In [2]:
import numpy as np

#
# Transfer functions
#
class TransferFunctions:
    def relu(x, derivative=False):
        if (derivative == True):
            for i in range(0, len(x)):
                for k in range(len(x[i])):
                    if x[i][k] > 0:
                        x[i][k] = 1
                    else:
                        x[i][k] = 0
            return x
        for i in range(0, len(x)):
            for k in range(0, len(x[i])):
                if x[i][k] > 0:
                    pass  # do nothing since it would be effectively replacing x with x
                else:
                    x[i][k] = 0
        return x

    def sgm(x, Derivative=False):
        if not Derivative:
            return 1.0 / (1.0 + np.exp(-x))
        else:
            #out = sgm(x)
            out = 1.0 / (1.0 + np.exp(-x))
            return out * (1.0 - out)
    
    def linear(x, Derivative=False):
        if not Derivative:
            return x
        else:
            return 1.0
    
    def gaussian(x, Derivative=False):
        if not Derivative:
            return np.exp(-x**2)
        else:
            return -2*x*np.exp(-x**2)
    
    def tanh(x, Derivative=False):
        if not Derivative:
            return np.tanh(x)
        else:
            return 1.0 - np.tanh(x)**2
    
    def truncLinear(x, Derivative=False):
        if not Derivative:
            y = x.copy()
            y[y < 0] = 0
            return y
        else:
            return 1.0
        
#
# Classes
#
class BackPropagationNetwork:
    """A back-propagation network"""
    
    #
    # Class methods
    #
    def __init__(self, layerSize, layerFunctions=None):
        """Initialize the network"""
        
        self.layerCount = 0
        self.shape = None
        self.weights = []
        self.tFuncs = []
        
        # Layer info
        self.layerCount = len(layerSize) - 1
        self.shape = layerSize
        
        if layerFunctions is None:
            lFuncs = []
            for i in range(self.layerCount):
                if i == self.layerCount - 1:
                    lFuncs.append(TransferFunctions.linear)
                else:
                    lFuncs.append(TransferFunctions.sgm)
        else:
            if len(layerSize) != len(layerFunctions):
                raise ValueError("Incompatible list of transfer functions.")
            elif layerFunctions[0] is not None:
                raise ValueError("Input layer cannot have a transfer function.")
            else:
                lFuncs = layerFunctions[1:]
        
        self.tFuncs = lFuncs
        
        # Data from last Run
        self._layerInput = []
        self._layerOutput = []
        self._previousWeightDelta = []
        
        # Create the weight arrays
        for (l1,l2) in zip(layerSize[:-1], layerSize[1:]):
            self.weights.append(np.random.normal(scale=0.01, size = (l2, l1+1)))
            self._previousWeightDelta.append(np.zeros((l2, l1+1)))
    
    #
    # Run method
    #
    def Run(self, input):
        """Run the network based on the input data"""
        
        lnCases = input.shape[0]
        
        # Clear out the previous intermediate value lists
        self._layerInput = []
        self._layerOutput = []
        
        # Run it!
        for index in range(self.layerCount):
            # Determine layer input
            if index == 0:
                layerInput = self.weights[0].dot(np.vstack([input.T, np.ones([1, lnCases])]))
            else:
                layerInput = self.weights[index].dot(np.vstack([self._layerOutput[-1], np.ones([1, lnCases])]))
            
            self._layerInput.append(layerInput)
            self._layerOutput.append(self.tFuncs[index](layerInput))
        
        return self._layerOutput[-1].T
                 
    #
    # TrainEpoch method
    #
    def TrainEpoch(self, input, target, trainingRate = 0.01, momentum = 0.7): #trainingRate = 0.02, momentum = 0.5
        """This method trains the network for one epoch"""
        
        delta = []
        lnCases = input.shape[0]
        
        # First run the network
        self.Run(input)
        
        # Calculate our deltas
        for index in reversed(range(self.layerCount)):
            if index == self.layerCount - 1:
                # Compare to the target values
                output_delta = self._layerOutput[index] - target.T
                error = np.sum(output_delta**2)
                delta.append(output_delta * self.tFuncs[index](self._layerInput[index], True))
            else:
                # Compare to the following layer's delta
                delta_pullback = self.weights[index + 1].T.dot(delta[-1])
                delta.append(delta_pullback[:-1, :] * self.tFuncs[index](self._layerInput[index], True))
            
        # Compute weight deltas
        for index in range(self.layerCount):
            delta_index = self.layerCount - 1 - index
            
            if index == 0:
                layerOutput = np.vstack([input.T, np.ones([1, lnCases])])
            else:
                layerOutput = np.vstack([self._layerOutput[index - 1], np.ones([1, self._layerOutput[index - 1].shape[1]])])
            
            curWeightDelta = np.sum(\
                                 layerOutput[None,:,:].transpose(2, 0 ,1) * delta[delta_index][None,:,:].transpose(2, 1, 0)\
                                 , axis = 0)
            
            weightDelta = trainingRate * curWeightDelta + momentum * self._previousWeightDelta[index]
            
            self.weights[index] -= weightDelta
            
            self._previousWeightDelta[index] = weightDelta
        
        return error
    
#normalization function
def norm(dizi):
    #global Maxs, Mins
    #Maxs = np.amax(dizi,axis=0)
    #Mins = np.amin(dizi,axis=0)
    A = []
    for d in dizi:
        a = []
        for i in range(d.shape[0]):
            new = (d[i]-Mins[i])/(Maxs[i]-Mins[i])
            a.append(new)
        A.append(a)
    New = np.array(A)
    return New
    
#sorgu yapma fonksiyonu
def Sor(inp):
    A = []
    a = []
    for i in range(inp.shape[0]):
        new = (inp[i]-Mins[i])/(Maxs[i]-Mins[i])
        a.append(new)
    A.append(a)
    New = np.array(A)
    print("normalize edildi: ",New)
    deger = bpn.Run(New)
    return deger

#
# If run as a script, create a test object
#
if __name__ == "__main__":
 
    #veri içeri alma
    from numpy import genfromtxt 
    my_data = genfromtxt('winequality-white.csv', delimiter=';',skip_header=1)
    print("dosyadan veri okundu...")
    #for normalizaiton
    global Maxs, Mins
    Maxs = np.amax(my_data,axis=0)
    Mins = np.amin(my_data,axis=0) 
    print("alt üst degerler saptandı...")
    #print(Maxs,Mins)
    
    ayrim = 500
    train = my_data[:ayrim,:] #my_data[:1200,:]
    test = my_data[ayrim:,:]
    train_input = train[:,:11]
    train_target= train[:,11:]
    test_input = test[:,:11]
    test_target = test[:, 11:]
    print("train test setleri ayrıldı. veri satır sayısı: ",train.shape )
    
    learning_rate = 0.03
    momentum = 0.5
    
    #lvTarget = norm(train_target)
    lvTarget = train_target/10
    lvInput = norm(train_input)
    #katman sayısı kadar Transfer Fonksiyonu EKLE
    lFuncs = [None, TransferFunctions.relu, TransferFunctions.sgm, TransferFunctions.sgm]
    #Katman yapısını belirle
    bpn = BackPropagationNetwork((11,15,3,1), lFuncs)
    print("Neural Network olusturuldu... Yapı: ",bpn.shape)
    #iterasyon sayısı ve istenilen hata oranı gir
    lnMax = 10000 #50000
    lnErr = 1e-1  #1e-6 0.1
    print("train basladı. epoch: ",lnMax," L_rate",learning_rate," momentum: ",momentum)
    for i in range(lnMax+1):
        err = bpn.TrainEpoch(lvInput, lvTarget,learning_rate, momentum)
        if i % 2000 == 0 and i > 0: #5000
            print("Iteration {0:6d}K - Error: {1:0.6f}".format(int(i/1000), err))
        if err <= lnErr:
            print("İstenilen hata oranına ulaşıldı. Iter: {0}".format(i))
            break     
    print("train tamamlandı...")   
        
    # Display output
    #lvOutput = bpn.Run(lvInput)
    #for i in range(train_input.shape[0]):
        #print("Input: {0} Output: {1} expect: {2}".format(train_input[i], lvOutput[i],lvTarget[i]))
     #   print("Output: {0} expect: {1}".format(lvOutput[i],lvTarget[i]))
#test safhası
    print("test başladı...")
    test_input_norm = norm(test_input)
    test_output = bpn.Run(test_input_norm)
    dogru = 0
    for i in range(test_output.shape[0]):
        if round(test_output[i][0],1) == test_target[i][0]/10:
            dogru = dogru + 1
    print("oran: ",100*dogru/i)

dosyadan veri okundu...
alt üst degerler saptandı...
train test setleri ayrıldı. veri satır sayısı:  (500, 12)
Neural Network olusturuldu... Yapı:  (11, 15, 3, 1)
train basladı. epoch:  10000  L_rate 0.03  momentum:  0.5
Iteration      2K - Error: 2.799276
Iteration      4K - Error: 2.778714
Iteration      6K - Error: 2.769847
Iteration      8K - Error: 2.771454
Iteration     10K - Error: 2.763819
train tamamlandı...
test başladı...
oran:  49.783943597907665


In [12]:
    test_input_norm = norm(test_input)
    test_output = bpn.Run(test_input_norm)
    dogru = 0
    print("test:",round(test_output[2][0] ,1)*10," target",test_target[2][0])
    for i in range(test_output.shape[0]):
        print(test_output[i],'',test_target[i])
        if round(test_output[i][0],1) == test_target[i][0]/10:
            dogru = dogru + 1
    print("oran: ",100*dogru/i)

test: 5.0  target 6.0
[0.50335822]  [6.]
[0.5362126]  [6.]
[0.51735145]  [6.]
[0.46801956]  [5.]
[0.46431724]  [5.]
[0.46801956]  [5.]
[0.51977517]  [5.]
[0.49213768]  [5.]
[0.50808416]  [6.]
[0.45515298]  [5.]
[0.50386912]  [5.]
[0.49371798]  [5.]
[0.49522076]  [5.]
[0.61684728]  [6.]
[0.50386912]  [5.]
[0.57684663]  [6.]
[0.53101141]  [6.]
[0.55275101]  [6.]
[0.67352498]  [6.]
[0.43950162]  [6.]
[0.50922895]  [5.]
[0.67352498]  [6.]
[0.53222572]  [5.]
[0.54680409]  [5.]
[0.47383282]  [5.]
[0.4322318]  [5.]
[0.41890051]  [5.]
[0.41508584]  [5.]
[0.55889724]  [7.]
[0.51327752]  [5.]
[0.47577367]  [5.]
[0.59586012]  [5.]
[0.59586012]  [5.]
[0.63965171]  [6.]
[0.46481368]  [6.]
[0.48033251]  [5.]
[0.48449954]  [5.]
[0.48742516]  [5.]
[0.48463638]  [5.]
[0.48311717]  [5.]
[0.48033251]  [5.]
[0.48449954]  [5.]
[0.84993445]  [6.]
[0.50059244]  [5.]
[0.84993445]  [6.]
[0.48977103]  [5.]
[0.47925843]  [5.]
[0.54024547]  [5.]
[0.59577091]  [6.]
[0.53324807]  [6.]
[0.48724]  [6.]
[0.42357288]  

[0.45101244]  [6.]
[0.54062941]  [6.]
[0.61846764]  [7.]
[0.52915105]  [6.]
[0.37084669]  [6.]
[0.70619959]  [6.]
[0.5102005]  [5.]
[0.83348171]  [5.]
[0.63406064]  [6.]
[0.54960213]  [5.]
[0.50170113]  [6.]
[0.37172806]  [5.]
[0.37172806]  [5.]
[0.71958139]  [6.]
[0.59703909]  [5.]
[0.76365397]  [5.]
[0.45452781]  [5.]
[0.45489966]  [5.]
[0.63194459]  [5.]
[0.64851398]  [6.]
[0.45370008]  [5.]
[0.45370008]  [5.]
[0.52465442]  [5.]
[0.52465442]  [5.]
[0.51447141]  [6.]
[0.46269213]  [5.]
[0.51447141]  [6.]
[0.6170543]  [5.]
[0.70485447]  [6.]
[0.46216903]  [4.]
[0.5941213]  [5.]
[0.62919927]  [5.]
[0.37968069]  [5.]
[0.37646081]  [5.]
[0.46851196]  [7.]
[0.61883222]  [6.]
[0.45979728]  [5.]
[0.40239486]  [5.]
[0.45979728]  [5.]
[0.40239486]  [5.]
[0.45979728]  [5.]
[0.52718478]  [7.]
[0.52959024]  [5.]
[0.57785129]  [4.]
[0.62509918]  [7.]
[0.78728033]  [6.]
[0.60462102]  [5.]
[0.42649267]  [5.]
[0.98189385]  [5.]
[0.53314663]  [6.]
[0.44681574]  [5.]
[0.50128538]  [5.]
[0.60462102]  [

[0.34042918]  [5.]
[0.61329884]  [7.]
[0.61329884]  [7.]
[0.61329884]  [7.]
[0.40491067]  [5.]
[0.61329884]  [7.]
[0.52975088]  [7.]
[0.54812421]  [6.]
[0.50949908]  [5.]
[0.54812421]  [6.]
[0.6557203]  [6.]
[0.79054582]  [6.]
[0.78339823]  [6.]
[0.48640757]  [6.]
[0.8460617]  [6.]
[0.59958649]  [6.]
[0.65696121]  [6.]
[0.71490028]  [6.]
[0.71490028]  [6.]
[0.45940736]  [6.]
[0.77410973]  [6.]
[0.84363146]  [6.]
[0.47025473]  [5.]
[0.5581146]  [5.]
[0.57005777]  [5.]
[0.58528891]  [7.]
[0.49194426]  [5.]
[0.72739421]  [6.]
[0.44637492]  [5.]
[0.49194426]  [5.]
[0.82544214]  [4.]
[0.5179492]  [6.]
[0.60616193]  [4.]
[0.6705174]  [6.]
[0.5179492]  [6.]
[0.60242544]  [4.]
[0.5694019]  [4.]
[0.47757754]  [5.]
[0.57439238]  [5.]
[0.64663313]  [6.]
[0.47292656]  [5.]
[0.9690265]  [6.]
[0.58144682]  [5.]
[0.49456581]  [5.]
[0.58144682]  [5.]
[0.63398731]  [6.]
[0.69739175]  [6.]
[0.69739175]  [6.]
[0.50566314]  [5.]
[0.55367605]  [5.]
[0.65750206]  [5.]
[0.63004492]  [5.]
[0.60757724]  [5.]
[

In [25]:
Sor(np.array([7.5, 0.52, 0.16, 1.9, 0.085, 12, 35, 0.9968, 3.38, 0.62, 9.5])) #7
#Sor(np.array([7.5, 0.52, 0.11, 1.5, 0.079, 11, 39, 0.9968, 3.42, 0.58, 9.6])) #5
#Sor(np.array([10.3, 0.32, 0.45, 6.4, 0.073, 5, 13, 0.9976, 3.23, 0.82, 12.6 ])) #8

normalize edildi:  [[0.25663717 0.2739726  0.16       0.06849315 0.12186978 0.15492958
  0.1024735  0.49412628 0.50393701 0.17365269 0.16923077]]


array([[0.54406619]])